In [1]:
import pandas as pd # 필요 라이브러리 불러오기
from datasketch import MinHashLSHForest, MinHash
from collections import Counter
import pytagcloud
import plotly
import plotly.graph_objs as go

In [2]:
# 데이터 불러오기
news = pd.read_csv("./korean_news_topic_201711.CSV", engine='python')
news = news['토픽키워드']
news = news.values.tolist()
keywords = [] # 뉴스 별 키워드 추출

for n in news:
    keywords.append(n.split(','))

In [24]:
m = [] # MinHash 객체를 각 뉴스 별로 만듦
for i in range(len(keywords)):
    m.append(MinHash(num_perm=128)) #num_perm = hash 수 많을수록 정확도 높음

for i, k in enumerate(keywords): # 각 단어를 min hashing 함
    for w in k:
        m[i].update(w.encode('utf8'))
        
# MinHash LSH Forest를 만듦 (MinHash값을 저장하는 tree)
forest = MinHashLSHForest(num_perm=128)

# 각 문서 별 MinHash 객체를 forest에 추가
for i, t_m in enumerate(m):
    forest.add(i, t_m)
forest.index()

# m[0] 문서와 비슷한 Top 11을 찾음
result = forest.query(m[0], 11)
news[0]

'연준,뉴욕증시,결정,강세,다우,금리,정책,다우지수,실업률,2시,변동성,중심,동결,전일,이달,통화정책,교수,영향,지명,2.5%,65.2,2만,전년,연방공개시장위원회,fed,개최,6.1%,지속,이틀,구매관리자지수,공개,12월,동부,계절,후반,화이자,상승세,거래일,전문가,nyse,제롬,0.3%,25bp,제약업체,기준,위원회,산업평균지수,runiron,페드워치,진단'

In [ ]:
if(0 in result):
    result.remove(0)
else:
    result = result[0:10]

In [15]:
keywords =[]
for n in result:
    keywords = keywords + news[n].split(',')
    
count = Counter(keywords) # 각 단어 별 개수 계산하기
tags = count.most_common(200) # 상위 200개 추출

word = []
counts = []

for key in tags:
    word.append(key[0])
    counts.append(key[1])

In [16]:
# histogram 만들기
data = [go.Bar(x=word, y=counts)]
filename = 'doc-histogram-lsh'
plotly.offline.plot(data, filename=filename + '.html') # 만들어진 histogram 저장

taglist = pytagcloud.make_tags(tags, maxsize=30) # wordcloud 만들기
pytagcloud.create_tag_image(taglist, 'doc-lsh.jpg', fontname='Malgun Gothic',
rectangular=False)